# Setup

In [1]:
!git clone https://github.com/onlookertozip/SamsungElectronics_Gumi.git

fatal: destination path 'SamsungElectronics_Gumi' already exists and is not an empty directory.


# Part2: RAG (Retrieval-Augmented Generation)
- 모델의 학습 데이터에 포함되지 않은 데이터를 사용 (환각 방지)
- **외부 데이터**를 검색(retrieval)한 후, 생성(generation) 단계에서 LLM에 전달
- 모델은 주어진 컨텍스트나 질문에 더 적합하고 풍부한 정보를 반영한 답변을 생성
- 논문: https://arxiv.org/abs/2005.11401

## 0. 환경 구성

### 1) 라이브러리 설치

# 설치 후 재시작 필요

In [ ]:
!pip install --upgrade openai

In [2]:
!pip install -q langchain langchain-openai langchain_community tiktoken chromadb

In [3]:
import langchain

langchain.__version__

'0.2.17'

### 2) OpenAI 인증키 설정

In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 1. RAG 파이프라인 개요
- Load Data - Text Split - Indexing - Retrieval - Generation



##### Step 1: Load Data

In [5]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

1
13171
 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.

문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.

명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대부분은 도입부 초반만 읽고 나가버리니까요. 각 정책 

##### Step 2: Split texts

In [6]:
# Text Split (Documents -> small chunks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])

18
page_content='제안과 채택
 백:아님 § 관료주의  문서를 참고하십시오. 단축백:제안
제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.
'제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.' metadata={'source': '

In [7]:
# page_content 속성
splits[10].page_content

'제안과 채택\n\xa0백:아님 §\xa0관료주의  문서를 참고하십시오. 단축백:제안\n제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.\n\'제안\'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.\n좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.\n정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.'

In [8]:
# metadata 속성
splits[10].metadata

{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8',
 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전',
 'language': 'ko'}

##### Step 3: Indexing

In [9]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")
print(len(docs))
print(docs[0].page_content)

4
격하
특정 정책이나 지침이 편집 관행이나 공동체 규범이 바뀌며 쓸모없어질 수 있고, 다른 문서가 개선되어 내용이 중복될 수 있으며, 불필요한 내용이 증식할 수도 있습니다. 이 경우 편집자들은 정책을 지침으로 격하하거나, 정책 또는 지침을 보충 설명, 정보문, 수필 또는 중단 문서로 격하할 것을 제안할 수 있습니다. 
격하 과정은 채택 과정과 비슷합니다. 일반적으로 토론 문서 내 논의가 시작되고 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다. 논의가 충분히 이루어진 후, 제3의 편집자가 토론을 종료하고 평가한 후 상태 변경 총의가 형성되었는지 판단해야 합니다. 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 정책/지침임을 알립니다.
소수의 공동체 인원만 지지하는 수필, 정보문 및 기타 비공식 문서는 일반적으로 주된 작성자의 사용자 이름공간으로 이동합니다. 이러한 논의는 일반적으로 해당 문서의 토론란에서 이루어지며, 간혹 위키백과:의견 요청을 통해 처리되기도 합니다.

같이 보기
위키백과:위키백과의 정책과 지침 목록
위키백과:의견 요청
수필

위키백과:제품, 절차, 정책
위키백과:위키백과 공동체의 기대와 규범
기타 링크


##### Step 4: Retrieval ~ Generation

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("격하 과정에 대해서 설명해주세요.")

'격하 과정은 특정 정책이나 지침이 더 이상 필요하지 않거나 개선이 필요한 경우에 해당 문서를 수정하거나 중단시키는 과정을 말합니다. 이를 위해 편집자들은 해당 정책이나 지침을 수정하거나 중단시키는 논의를 시작하고, 공동체의 참여를 요청합니다. 논의가 충분히 이루어진 후, 제3의 편집자가 토론을 종료하고 평가한 후 상태 변경 총의가 형성되었는지 판단합니다. 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 것임을 알립니다. 격하 과정은 채택 과정과 유사하며, 공동체의 강력한 지지를 바탕으로 진행됩니다.'

### PDF 파일기반 질의응답 챗봇 (랭체인, 그라디오, ChatGPT)

# 설치 후 재시작 필요

In [1]:
!pip install openai # openai 라이브러리를 설치합니다.
!pip install langchain # 랭체인 라이브러리를 설치합니다.

In [2]:
!pip install pypdf # pdf 로딩용
!pip install chromadb # 벡터스토어
!pip install tiktoken # 토큰 계산용

In [7]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [4]:
loader = PyPDFLoader("SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [5]:
texts

[Document(metadata={'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf', 'page': 0}, page_content='- 1 -\n2023년 가을 학술대회 및 제41회 정기총회 안내\n▢ 행사개요\n  ○ 행사명 : 2023년 가을 학술대회 및 제41회 정기총회\n  ○  기   간  :  2 0 2 3 .  1 0 .  2 5 . ( 수 )  ~  2 7 . ( 금 )  \n  ○ 장  소 : 여수 베네치아 호텔&리조트\n             \n▢ 초록 및 프로시딩 접수   \n  ○  접수기간 : 2023. 8. 21.(월) ~ 9. 15.(금)\n  ○ 접수방법 : 홈페이지 -> 학술대회 (초록제출을 위해서는 연회비납부 및 사전등록 필요)\n  ○ 영문초록양식 : 영문초록. 저자명 풀 네임 사용\n  ○ 발표분야 : 특별세션 구성하면서 추가될 수 있음.\n-태양 및 우주환경 (태양, 자기권, 전리층, 고층대기, COSMIC RAY 등 및 관련 관측 장비)\n-우주응용 (측지, 마이크로중력 환경 실험, 원격탐사 및 관련 탑재체)\n-우주기술 (위성운영, 위성체) \n-우주천문 (우주천문 탑재체, 천문 관측기기, 천문학 일반, 고천문, 천문대운영, 소프트웨어) \n-우주감시 (광학 및 레이다 관측, 우주물체궤도 분석, 우주위험 분석) \n-달과 우주탐사: 과학과 기술 그리고 정책 (태양계 탐사, 달탐사)\n-초소형 위성\n-안보우주\n-지상 및 우주 인프라 운영기술\n-특별세션 : Open New Horizon with L4 mission\n▢ 발표자  \n   ○ 논문 제출자는 발표자로 간주하며 제1저자이어야하며 정회원이고 연회비를 완납하여야 함.\n   ○ 초록 마감 후에는 발표형식(구두, 포스터) 조정이 어려우므로 접수시 정확히 선택.\n   ○ 2편 발표자는 구두발표 2편 or 구두발표 1편 + 포스터발표 1편 (포스터 발표만 2편 불가) \n   ○ 발표자료

In [8]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(texts, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True)

<ipython-input-9-8aec39d14a1e>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4


In [10]:
query = "음료 지원은?"
result = chain(query)
print(result)

<ipython-input-10-c76d6a11b032>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = chain(query)


{'question': '음료 지원은?', 'answer': '음료 지원은 없습니다.\n', 'sources': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회 세부안내.pdf', 'source_documents': [Document(metadata={'page': 1, 'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf'}, page_content='- 2 -\n  ○ 등록비\n     \n구분 정회원 군경/학부생/\n 전일제 대학원생 비회원\n사전등록비 270,000원 150,000원 320,000\n현장등록비 300,000원 180,000원 320,000\n  ※ 연구소, 또는 기타 기관에서 등록비 지원을 받는 대학원생은 정회원 등록비.(학생 등록비 아님)\n  ○ 식사 : 온라인 신청기간 10월 4일 ~ 13일까지\n    \n구분 조식, 중식(1회 기준) 만찬\n식사비 5,000원 신청자에 한해 지원\n(일부학회지원)\n만찬은 신청한 회원에 한해 지원되며  그 외 식사는 회당 5,000원(25,000원 상당)\n씩 회원 부담입니다.\n등록시 식사 신청을 하지 않은 분들은 현장에서 일반 식사비를 지불해야 합니다.\n가족식사 신청 만찬 2만원 / 26일 중식 1만원  / 조식뷔폐는 가족식권 판매 안함\n(단, 현금 결제, 카드결제 안됨)\n  ○ 지정숙박 : 개별 신청 (신청서 다운로드)\n      \n25일(수), 26일(목) \n 더블 : 100,000원  / 트윈 : 110,000원  /  페밀리 트윈 : 120,000원\n▢ 행사진행 도우미 모집\n  ○ 모집인원 : 00명 \n  ○ 신청기간 : 2023. 10. 2.(월)까지. 심사 후 통보\n  ○ 신청방법 : 신청서 작성하여 신청 (이메일: ksss@ksss.or.kr, 팩스 042-865-3392)\n  ○ 신청서 : 학회 홈페이지 자료실\n  ○ 인건비 : 300,000원 (숙박,

In [11]:
result['answer']

'음료 지원은 없습니다.\n'

In [12]:
result['sources']

'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회 세부안내.pdf'

In [13]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean and in Markdown format:"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": prompt}

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [15]:
query = "음료 지원은?"
result = chain(query)
print(result)

{'question': '음료 지원은?', 'answer': '**음료 지원은 없습니다.**  \n**', 'sources': '**', 'source_documents': [Document(metadata={'page': 1, 'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf'}, page_content='- 2 -\n  ○ 등록비\n     \n구분 정회원 군경/학부생/\n 전일제 대학원생 비회원\n사전등록비 270,000원 150,000원 320,000\n현장등록비 300,000원 180,000원 320,000\n  ※ 연구소, 또는 기타 기관에서 등록비 지원을 받는 대학원생은 정회원 등록비.(학생 등록비 아님)\n  ○ 식사 : 온라인 신청기간 10월 4일 ~ 13일까지\n    \n구분 조식, 중식(1회 기준) 만찬\n식사비 5,000원 신청자에 한해 지원\n(일부학회지원)\n만찬은 신청한 회원에 한해 지원되며  그 외 식사는 회당 5,000원(25,000원 상당)\n씩 회원 부담입니다.\n등록시 식사 신청을 하지 않은 분들은 현장에서 일반 식사비를 지불해야 합니다.\n가족식사 신청 만찬 2만원 / 26일 중식 1만원  / 조식뷔폐는 가족식권 판매 안함\n(단, 현금 결제, 카드결제 안됨)\n  ○ 지정숙박 : 개별 신청 (신청서 다운로드)\n      \n25일(수), 26일(목) \n 더블 : 100,000원  / 트윈 : 110,000원  /  페밀리 트윈 : 120,000원\n▢ 행사진행 도우미 모집\n  ○ 모집인원 : 00명 \n  ○ 신청기간 : 2023. 10. 2.(월)까지. 심사 후 통보\n  ○ 신청방법 : 신청서 작성하여 신청 (이메일: ksss@ksss.or.kr, 팩스 042-865-3392)\n  ○ 신청서 : 학회 홈페이지 자료실\n  ○ 인건비 : 300,000원 (숙박, 식사 제공)\n  ○ 참여일 : 2023. 10. 25.(수) - 27.(금) 

In [16]:
result['answer']

'**음료 지원은 없습니다.**  \n**'

In [17]:
result['source_documents']

[Document(metadata={'page': 1, 'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf'}, page_content='- 2 -\n  ○ 등록비\n     \n구분 정회원 군경/학부생/\n 전일제 대학원생 비회원\n사전등록비 270,000원 150,000원 320,000\n현장등록비 300,000원 180,000원 320,000\n  ※ 연구소, 또는 기타 기관에서 등록비 지원을 받는 대학원생은 정회원 등록비.(학생 등록비 아님)\n  ○ 식사 : 온라인 신청기간 10월 4일 ~ 13일까지\n    \n구분 조식, 중식(1회 기준) 만찬\n식사비 5,000원 신청자에 한해 지원\n(일부학회지원)\n만찬은 신청한 회원에 한해 지원되며  그 외 식사는 회당 5,000원(25,000원 상당)\n씩 회원 부담입니다.\n등록시 식사 신청을 하지 않은 분들은 현장에서 일반 식사비를 지불해야 합니다.\n가족식사 신청 만찬 2만원 / 26일 중식 1만원  / 조식뷔폐는 가족식권 판매 안함\n(단, 현금 결제, 카드결제 안됨)\n  ○ 지정숙박 : 개별 신청 (신청서 다운로드)\n      \n25일(수), 26일(목) \n 더블 : 100,000원  / 트윈 : 110,000원  /  페밀리 트윈 : 120,000원\n▢ 행사진행 도우미 모집\n  ○ 모집인원 : 00명 \n  ○ 신청기간 : 2023. 10. 2.(월)까지. 심사 후 통보\n  ○ 신청방법 : 신청서 작성하여 신청 (이메일: ksss@ksss.or.kr, 팩스 042-865-3392)\n  ○ 신청서 : 학회 홈페이지 자료실\n  ○ 인건비 : 300,000원 (숙박, 식사 제공)\n  ○ 참여일 : 2023. 10. 25.(수) - 27.(금) \n▢ 학생등록비 지원(선착순 3명)\n  ○  신청자격 : 발표자(공동발표자) 중 연구과제에서 등록비를 지원받을 수 없는 학부생과 정회  \n     원 중 전일제 석, 박

In [18]:
for doc in result['source_documents']:
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))

내용 : - 2 -   ○ 등록비       구분 정회원 군경/학부생/  전일제 대학원생 비회원 사전등록비 270,000원 150,000원 320,000 현장등록비 300,000원 180,
파일 : SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf
페이지 : 1
내용 : - 3 -    ○ 신청서 : 학회 홈페이지 자료실 ▢ 제41차 정기총회 개최    ○  일 시 :  2 0 2 3 .  1 0 .  2 6 . ( 목 )    ○ 주요안건: 20
파일 : SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf
페이지 : 2


# 설치 후 재시작 필요

In [19]:
!pip install gradio # 그라디오 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.19
    Uninstalling pydantic-1.10.19:
      Successfully uninstalled pydantic-1.10.19


In [1]:
import gradio as gr

In [ ]:
def respond(message, chat_history):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.

    result = chain(message)

    bot_message = result['answer']

    for i, doc in enumerate(result['source_documents']):
        bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '

    chat_history.append((message, bot_message))  # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.

    return "", chat_history  # 수정된 채팅 기록을 반환합니다.

with gr.Blocks() as demo:  # gr.Blocks()를 사용하여 인터페이스를 생성합니다.
    chatbot = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    msg.submit(respond, [msg, chatbot], [msg, chatbot])  # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.

demo.launch(debug=True)  # 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fa552e1c62a08b5ee0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
